# Main

In [7]:
#!pip install ray[rllib,tune]
#!pip install pettingzoo pygame pymunk
#!pip install torch
#!pip install wandb

In [1]:
from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec
#from ray.rllib.core.rl_module.rl_module import SingleAgentRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-20 20:12:29,233	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-20 20:12:31,361	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-10-20 20:12:32,405	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.core.rl_module.marl_module` has been deprecated. Use `ray.rllib.core.rl_module.multi_rl_module` instead. This will raise an error in the future!


# Restoring Weights

In [2]:
import sys
from glob import glob
from os import path
import ray
from ray.rllib.policy.policy import Policy

ray.shutdown() 

# Trained to about 0 combined return
#checkpoint_path = "/root/ray_results/PPO_2024-08-28_20-57-45/PPO_2_agent_env_2cf59_00000_0_2024-08-28_20-57-45/checkpoint_000000"
checkpoint_path = "/root/ray_results/PPO_2024-10-20_17-36-00/PPO_2_agent_waterworld_c5957_00000_0_2024-10-20_17-36-00/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}
#specs = {path.basename(p) : SingleAgentRLModuleSpec(load_state_path=p) for p in pols} # Non-deterministic policy weight return (implies new)


num_agents = 2

register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}


resto_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        #model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            #load_state_path=
            #module_specs=specs,
            module_specs={p: RLModuleSpec() for p in policies},
            #module_specs={p: Policy.from_checkpoint(pols[0]) for p in policies},
        ),
    )
    .evaluation(
        evaluation_interval=1,
    )
)

resto_algo = resto_config.build()
#resto_algo.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
#resto_algo.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())
#resto_algo.get_policy("pursuer_0").get_weights()


/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2024-10-20 20:12:35,727	WARNING algorithm_config.py:4225 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py

In [ ]:
resto_algo.save_checkpoint("/tmp/temp_checkpoints")

In [6]:
specs['pursuer_0'].get_weights()

{'_logits._model.0.weight': array([[-0.0057713 ,  0.02130763, -0.00332906, ..., -0.00251019,
          0.01308718, -0.01480941],
        [ 0.02027893, -0.00918621,  0.0372359 , ...,  0.00163348,
          0.01085026,  0.00847802],
        [-0.03894725,  0.07293524, -0.01994372, ...,  0.0666676 ,
         -0.03469629, -0.02116322],
        [ 0.00803491,  0.02462115, -0.01157856, ..., -0.0220483 ,
         -0.02909413,  0.0477805 ]], dtype=float32),
 '_logits._model.0.bias': array([ 0.00049785,  0.00283682, -0.01044037, -0.01154366], dtype=float32),
 '_hidden_layers.0._model.0.weight': array([[-0.06341984, -0.01866287,  0.04820696, ..., -0.06329054,
         -0.0094757 , -0.1880102 ],
        [ 0.04908092, -0.02408476,  0.01319859, ...,  0.0380294 ,
          0.11712942,  0.10649226],
        [ 0.00440866, -0.02424742, -0.0211874 , ..., -0.00685773,
          0.04866119, -0.04014146],
        ...,
        [ 0.05629121,  0.01958698, -0.01932536, ..., -0.06350598,
          0.00073303,  0.

Above, we restore an algo to its original size, and evaluate below.

In [7]:
resto_algo.evaluate()

{'env_runners': {'episode_reward_max': np.float64(104.48447046235398),
  'episode_reward_min': np.float64(-49.97758701259906),
  'episode_reward_mean': np.float64(34.31495169008534),
  'episode_len_mean': np.float64(500.0),
  'episode_media': {},
  'episodes_timesteps_total': 5000,
  'policy_reward_min': {'pursuer_0': np.float64(-27.80399612520721),
   'pursuer_1': np.float64(-74.73581968384396)},
  'policy_reward_max': {'pursuer_0': np.float64(108.50397211584374),
   'pursuer_1': np.float64(66.59594375510522)},
  'policy_reward_mean': {'pursuer_0': np.float64(32.97170784631929),
   'pursuer_1': np.float64(1.3432438437660394)},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [np.float64(44.487376281462424),
    np.float64(98.65380257932057),
    np.float64(84.50964155764572),
    np.float64(20.08502968193733),
    np.float64(104.48447046235398),
    np.float64(-38.577462399422544),
    np.float64(-49.97758701259906),
    np.float64(20.465628189378837),
    np.float64(91.2633

Next, lets try with a different number of test agents.

In [6]:
import sys
from glob import glob
from os import path
from ray.rllib.policy.policy import Policy
import numpy as np

ray.shutdown()

# Trained to about 0 combined return
checkpoint_path = "/root/ray_results/PPO_2024-08-28_20-57-45/PPO_2_agent_env_2cf59_00000_0_2024-08-28_20-57-45/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}
#specs = {path.basename(p) : SingleAgentRLModuleSpec(load_state_path=p) for p in pols} # Non-deterministic policy weight return (implies new)

num_trained_agents = 2
num_test_agents = 6

register_env(f"{num_test_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_test_agents)))
policies = {f"pursuer_{i}" for i in range(num_test_agents)}


resto_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_test_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        #model_config_dict={"vf_share_layers": True},
        rl_module_spec=MultiAgentRLModuleSpec(
            #load_state_path=
            #module_specs=specs,
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
    .evaluation(
        evaluation_interval=1,
    )
)

resto_algo = resto_config.build()
for test_id in range(num_test_agents):
    train_id = np.random.randint(num_trained_agents)
    resto_algo.get_policy(f"pursuer_{test_id}").set_weights(specs[f"pursuer_{train_id}"].get_weights())

#resto_algo.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
#resto_algo.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())
#resto_algo.get_policy("pursuer_0").get_weights()

resto_algo.evaluate()

NameError: name 'ray' is not defined

Test for training:

In [4]:
resto_algo.train()

2024-09-03 17:55:20,672	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


{'evaluation': {'env_runners': {'episode_reward_max': np.float64(-364.769848341295),
   'episode_reward_min': np.float64(-619.0375683355406),
   'episode_reward_mean': np.float64(-493.78093420107905),
   'episode_len_mean': np.float64(500.0),
   'episode_media': {},
   'episodes_timesteps_total': 5000,
   'policy_reward_min': {'pursuer_0': np.float64(-113.90485384019159),
    'pursuer_1': np.float64(-109.27251741852379),
    'pursuer_2': np.float64(-114.55402342215696),
    'pursuer_3': np.float64(-111.76090622319133),
    'pursuer_4': np.float64(-109.0597747992956),
    'pursuer_5': np.float64(-217.50166684483233)},
   'policy_reward_max': {'pursuer_0': np.float64(11.303415129637894),
    'pursuer_1': np.float64(0.2069972847033158),
    'pursuer_2': np.float64(-46.84567547040106),
    'pursuer_3': np.float64(-6.933874989835202),
    'pursuer_4': np.float64(-7.903867561959164),
    'pursuer_5': np.float64(-102.74657319368887)},
   'policy_reward_mean': {'pursuer_0': np.float64(-62.6888

## Testing Callbacks



In [12]:
import ray
from ray.rllib.algorithms.callbacks import DefaultCallbacks

from typing import Dict, Tuple
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import Episode, RolloutWorker
from ray.rllib.policy import Policy

import numpy as np


class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, episode: Episode, **kwargs) -> None:
        episode.hist_data["mean_agent_return"] = []

    def on_episode_end(self, *, episode: Episode, **kwargs) -> None:
        episode.hist_data["mean_agent_return"].append(
            episode.total_reward / len(policies) )
        #episode.hist_data['mod_return'] = episode.hist_data['episode_reward']/2
    
    def on_train_result(self, *, algorithm, result: dict, **kwargs) -> None:
        #n_agents = len(policies)        
        n_agents = len(result['info']['learner'])
        result["num_agents"] = n_agents
        #result["mod_return"] = result['info']['hist_stats']["episode_reward"]/2
        result["mod_return"] = np.divide(
            result['env_runners']['hist_stats']['episode_reward'], n_agents)


ray.shutdown()


num_agents = 2
register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))
policies = {f"pursuer_{i}" for i in range(num_agents)}

config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        # Exact 1:1 mapping from AgentID to ModuleID.
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        rl_module_spec=MultiAgentRLModuleSpec(
            module_specs={p: SingleAgentRLModuleSpec() for p in policies},
        ),
    )
    #.evaluation(
    #    evaluation_interval=1,
    #)
    .callbacks(MyCallbacks)
    #.training(trained_agents=num_agents)
)

algo = config.build()

2024-09-28 02:04:18,759	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:557: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/conda/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.

(raylet) [2024-09-28 02:50:27,373 E 196176 196176] (raylet) node_manager.cc:3064: 3 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: e236b453d486271274e658140a30de7353a58d29b8000dc1df989521, IP: 10.42.142.216) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 10.42.142.216`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) 
(raylet) [2024-09-28 02:51:30,075 E 196176 196176] (raylet) node_manager.cc:3064: 7 Workers (tasks / ac

In [13]:
results = algo.train()

2024-09-28 02:05:00,769	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


In [38]:
results['env_runners']['hist_stats']['episode_reward']
np.divide(results['env_runners']['hist_stats']['episode_reward'],2)


[np.float64(-224.56955954147745),
 np.float64(-210.60704272678404),
 np.float64(-270.4674606087424),
 np.float64(-300.55186835564245),
 np.float64(-269.2034552824137),
 np.float64(-225.00378578885667),
 np.float64(-221.94251529451898),
 np.float64(-224.5826295071694),
 np.float64(-225.01705161938244),
 np.float64(-289.68626249930134),
 np.float64(-244.16540430415856),
 np.float64(-266.06146701715795),
 np.float64(-177.1772171119893),
 np.float64(-185.088584697531),
 np.float64(-243.36592972517653),
 np.float64(-225.87668550926975)]

In [45]:
len(results['info']['learner'])

2

## Async Script Runner

Async Running
`apt-get install tmux`
`tmux new-session -d 'python run_this.py'`

Other possible useful commands?
   `tmux ls`
   `tmux kill-session -t <session-number>`
   `tmux attach-session -t <session-number>`


tmux new-session -d 'python test.py --num-agents=2 --stop-iters=10'

tmux new-session -d 'python test.py --wandb-project=mini_test_ww2 --wandb-key=913528a8e92bf601b6eb055a459bcc89130c7f5f --num-env-runners=30 --checkpoint-at-end --num-agents=2'

## Plateau Stopper

In [ ]:
from ray.tune.stopper import TrialPlateauStopper

TrialPlateauStopper(metric="episode_return_mean")

run_rllib_example_script_experiment(base_config, args, scheduler=None)

In [ ]:
python test.py --wandb-project=mini_test_ww2 --wandb-key=913528a8e92bf601b6eb055a459bcc89130c7f5f \
--num-env-runners=30 --checkpoint-at-end --num-agents=2

## Organized Output

In [3]:
parser = add_rllib_example_script_args(
    default_iters=10,
    default_timesteps=1000000,
    default_reward=300,
)
args = parser.parse_args(args=[])

args.checkpoint_at_end = True
args.num_samples = 2
args.num_agents = 2
env = 'waterworld'

from pettingzoo.sisl import waterworld_v4
register_env(f"{args.num_agents}_agent_{env}", lambda _:
    ParallelPettingZooEnv(
        waterworld_v4.parallel_env(n_pursuers=args.num_agents)))
policies = {f"pursuer_{i}" for i in range(args.num_agents)}

config = (
        get_trainable_cls(args.algo)
        .get_default_config()
        .environment( f"{args.num_agents}_agent_{env}" )
        .multi_agent(
            policies=policies,
            # Exact 1:1 mapping from AgentID to ModuleID.
            policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
        )
        .rl_module(
            model_config_dict={"vf_share_layers": True},
            rl_module_spec=MultiAgentRLModuleSpec(
                module_specs={p: SingleAgentRLModuleSpec() for p in policies},
            ),
        )
    )

results = run_rllib_example_script_experiment(config,args)

/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2024-09-28 14:41:08,361	WARNING services.py:2017 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67047424 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/

== Status ==
Current time: 2024-09-28 14:41:10 (running for 00:00:00.19)
Using FIFO scheduling algorithm.
Logical resource usage: 0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-28_14-41-06_816937_376672/artifacts/2024-09-28_14-41-10/PPO_2024-09-28_14-41-10/driver_artifacts
Number of trials: 2/2 (2 PENDING)
+------------------------------------+----------+-------+
| Trial name                         | status   | loc   |
|------------------------------------+----------+-------|
| PPO_2_agent_waterworld_b4113_00000 | PENDING  |       |
| PPO_2_agent_waterworld_b4113_00001 | PENDING  |       |
+------------------------------------+----------+-------+




(PPO pid=408432) 2024-09-28 14:41:15,109	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.


== Status ==
Current time: 2024-09-28 14:41:15 (running for 00:00:05.25)
Using FIFO scheduling algorithm.
Logical resource usage: 6.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-28_14-41-06_816937_376672/artifacts/2024-09-28_14-41-10/PPO_2024-09-28_14-41-10/driver_artifacts
Number of trials: 2/2 (2 PENDING)
+------------------------------------+----------+-------+
| Trial name                         | status   | loc   |
|------------------------------------+----------+-------|
| PPO_2_agent_waterworld_b4113_00000 | PENDING  |       |
| PPO_2_agent_waterworld_b4113_00001 | PENDING  |       |
+------------------------------------+----------+-------+


== Status ==
Current time: 2024-09-28 14:41:20 (running for 00:00:10.28)
Using FIFO scheduling algorithm.
Logical resource usage: 6.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-28_14-41-06_816937_376672/artifacts/2024-09-28_14-41-10/PPO_2024-09-28_14-41-10/driver_artifacts
Number of trials: 2/2 (2 PENDING)
+---

(PPO pid=408432) Install gputil for GPU system monitoring.
(PPO pid=408432) 2024-09-28 14:41:21,595	WARNING algorithm_config.py:4258 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`. [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


...

(PPO pid=408432) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/PPO_2024-09-28_14-41-10/PPO_2_agent_waterworld_b4113_00001_1_2024-09-28_14-41-10/checkpoint_000000)
(PPO pid=408431) 2024-09-28 14:41:31,681	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!


== Status ==
Current time: 2024-09-28 14:45:28 (running for 00:04:17.47)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-28_14-41-06_816937_376672/artifacts/2024-09-28_14-41-10/PPO_2024-09-28_14-41-10/driver_artifacts
Number of trials: 2/2 (1 RUNNING, 1 TERMINATED)
+------------------------------------+------------+----------------------+--------+------------------+-------+-------------------+
| Trial name                         | status     | loc                  |   iter |   total time (s) |    ts |   combined return |
|------------------------------------+------------+----------------------+--------+------------------+-------+-------------------|
| PPO_2_agent_waterworld_b4113_00000 | RUNNING    | 10.42.142.216:408431 |      9 |          244.106 | 36000 |          -231.656 |
| PPO_2_agent_waterworld_b4113_00001 | TERMINATED | 10.42.142.216:408432 |     10 |          241.336 | 40000 |          -213.682 |
+-------

2024-09-28 14:45:53,345	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/PPO_2024-09-28_14-41-10' in 0.0250s.


== Status ==
Current time: 2024-09-28 14:45:53 (running for 00:04:42.68)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/64 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2024-09-28_14-41-06_816937_376672/artifacts/2024-09-28_14-41-10/PPO_2024-09-28_14-41-10/driver_artifacts
Number of trials: 2/2 (1 RUNNING, 1 TERMINATED)
+------------------------------------+------------+----------------------+--------+------------------+-------+-------------------+
| Trial name                         | status     | loc                  |   iter |   total time (s) |    ts |   combined return |
|------------------------------------+------------+----------------------+--------+------------------+-------+-------------------|
| PPO_2_agent_waterworld_b4113_00000 | RUNNING    | 10.42.142.216:408431 |     10 |          270.649 | 40000 |          -228.946 |
| PPO_2_agent_waterworld_b4113_00001 | TERMINATED | 10.42.142.216:408432 |     10 |          241.336 | 40000 |          -213.682 |
+-------

(PPO pid=408431) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/PPO_2024-09-28_14-41-10/PPO_2_agent_waterworld_b4113_00000_0_2024-09-28_14-41-10/checkpoint_000000)
2024-09-28 14:45:54,019	INFO tune.py:1041 -- Total run time: 283.44 seconds (282.72 seconds for the tuning loop).


In [4]:
results

ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(3.6451383), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss': np.float64(4.388929148266713), 'policy_loss': np.float64(-0.021291275953505343), 'vf_loss': np.float64(4.408763808508714), 'vf_explained_var': np.float64(0.02267032687862714), 'kl': np.float64(0.014566012486125185), 'entropy': np.float64(2.242644399901231), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(125.0), 'num_grad_updates_lifetime': np.float64(9120.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)}, 'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(2.6652873), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-0

In [11]:
#results.__getitem__(0).path
#results.__getitem__(0).metrics#['env_runners']['episode_reward_mean']
results[0]

Result(
  metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'pursuer_0': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(3.6451383), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss': np.float64(4.388929148266713), 'policy_loss': np.float64(-0.021291275953505343), 'vf_loss': np.float64(4.408763808508714), 'vf_explained_var': np.float64(0.02267032687862714), 'kl': np.float64(0.014566012486125185), 'entropy': np.float64(2.242644399901231), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(125.0), 'num_grad_updates_lifetime': np.float64(9120.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(479.5)}, 'pursuer_1': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(2.6652873), 'cur_kl_coeff': np.float64(0.10000000000000002), 'cur_lr': np.float64(5.0000000000000016e-05), 'total_loss':

In [ ]:
# move checkpoint contents:
import shutil 


for i in range(len(results)):   # or args.num_agents
    source = results.__getitem__(i).path
    score = results.__getitem__(i).metrics['env_runners']['episode_reward_mean']
    destination = f"/root/test/{env}/{algo}/{num_agents}/{score}"





shutil.move(source, destination, copy_function = shutil.copytree) 




## Pooled Restoration

In [18]:
from glob import glob
from os import path
import numpy as np

path_to_checkpoint = "/root/test/waterworld/PPO/2_agent/"

top_per = 0.5

x = glob(path_to_checkpoint+"*")
#print(x)
x.sort()
#print(x)
top_num = int(len(x)*top_per)
print(top_num)
#print(x[-top_num:])
candidate_runs = x[-top_num:]
#print(candidate_runs)
pool = np.array([glob(f+"/policies/*") for f in candidate_runs]).flatten()
pool

4


array(['/root/test/waterworld/PPO/2_agent/22.30845564138441/policies/pursuer_0',
       '/root/test/waterworld/PPO/2_agent/22.30845564138441/policies/pursuer_1',
       '/root/test/waterworld/PPO/2_agent/33.04836291061455/policies/pursuer_0',
       '/root/test/waterworld/PPO/2_agent/33.04836291061455/policies/pursuer_1',
       '/root/test/waterworld/PPO/2_agent/53.00706844190301/policies/pursuer_0',
       '/root/test/waterworld/PPO/2_agent/53.00706844190301/policies/pursuer_1',
       '/root/test/waterworld/PPO/2_agent/65.31630841725516/policies/pursuer_0',
       '/root/test/waterworld/PPO/2_agent/65.31630841725516/policies/pursuer_1'],
      dtype='<U70')

In [18]:
a = lambda x: [_ for _ in range(x)]

b = a(4)
b

[0, 1, 2, 3]

In [51]:
import numpy as np

n = 5

h = []
while len(h) < n:
    for e in np.random.choice(b, min(len(b),n-len(h)), replace=False):
        h.append(int(e))

h

[1, 3, 2, 0, 1]

# Retrain

In [15]:
import wandb
import numpy as np
from collections import deque

# Initialize Weights and Biases (optional)
use_wandb = True  # Set to False if you don't want to use Wandb
if use_wandb:
    wandb.init(project="your_project_name", config={
        "num_iterations": 100,
        "checkpoint_freq": 10,
        "evaluation_freq": 5,
        "max_episodes": 1000,
        "max_timesteps": 100000,
        "moving_average_window": 10,
        "reward_threshold": 200,
        "patience": 10,
        "min_reward_improvement": 1e-3
    })

# Assuming algo is already initialized (e.g., PPO or any other algorithm)
# algo = PPOConfig().environment("your_env").framework("torch").build()

num_iterations = 100  # Set your desired number of iterations
checkpoint_freq = 10   # How often to save checkpoints
evaluation_freq = 5    # How often to evaluate

# Stopping criteria
max_episodes = 1000    
max_timesteps = 100000  
reward_threshold = 200  
moving_average_window = 10  
patience = 10  
min_reward_improvement = 1e-3  

# Tracking variables
total_episodes, total_timesteps, no_improvement_steps = 0, 0, 0
best_moving_average_reward = -np.inf  
reward_history = deque(maxlen=moving_average_window)  # Efficient sliding window

def should_stop_early(moving_average_reward):
    """Check all stopping criteria."""
    if total_episodes >= max_episodes:
        print(f"Stopping due to max_episodes: {total_episodes}")
        return True
    if total_timesteps >= max_timesteps:
        print(f"Stopping due to max_timesteps: {total_timesteps}")
        return True
    if moving_average_reward >= reward_threshold:
        print(f"Stopping due to reward threshold: {moving_average_reward}")
        return True
    if no_improvement_steps >= patience:
        print(f"Stopping due to lack of improvement for {patience} iterations.")
        return True
    return False

for i in range(num_iterations):
    # Run one iteration of training
    results = algo.train()
    
    # Update tracking variables
    total_episodes += results['episodes_this_iter']
    total_timesteps += results['timesteps_this_iter']
    
    # Track and compute moving average reward
    
    reward_history.append(results['episode_reward_mean'])
    moving_average_reward = np.mean(reward_history)

    # Log important metrics to Wandb
    if use_wandb:
        wandb.log({
            "iteration": i,
            "moving_average_reward": moving_average_reward,
            "episode_reward_mean": results['episode_reward_mean'],
            "total_episodes": total_episodes,
            "total_timesteps": total_timesteps
        })

    # Print iteration summary
    print(f"Iteration {i}: reward_mean = {results['episode_reward_mean']}, "
          f"moving_average_reward = {moving_average_reward}, "
          f"total_episodes = {total_episodes}, total_timesteps = {total_timesteps}")

    # Save a checkpoint at intervals
    if i % checkpoint_freq == 0:
        checkpoint = algo.save(checkpoint_dir="checkpoints")
        print(f"Checkpoint saved at iteration {i}: {checkpoint}")

    # Evaluate at intervals
    if i % evaluation_freq == 0:
        eval_results = algo.evaluate()
        if use_wandb:
            wandb.log(eval_results)
        print(f"Evaluation results at iteration {i}: {eval_results}")

    # Check if moving average reward has improved
    if moving_average_reward > best_moving_average_reward + min_reward_improvement:
        best_moving_average_reward = moving_average_reward
        no_improvement_steps = 0  # Reset patience counter
    else:
        no_improvement_steps += 1  # Increment patience counter

    # Check stopping conditions
    if should_stop_early(moving_average_reward):
        break
        wandb.log({"termination_reason": term})


# Finish the Wandb run (optional)
if use_wandb:
    wandb.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bhosley (no-organization-for-signup). Use `wandb login --relogin` to force relogin


NameError: name 'algo' is not defined

In [ ]:
from ray.tune import Trainable, Tuner
from ray.tune.registry import register_trainable

print(issubclass(type("string"), Trainable))
print(issubclass(type(resto_algo), Trainable))
print(type(resto_algo))
resto_algo.config = resto_algo.config
register_trainable("cloned_algo", lambda _: resto_algo)
test_tuner = Tuner("cloned_algo")

False
True
<class 'ray.rllib.algorithms.ppo.ppo.PPO'>


In [ ]:
from ray.tune import Trainable, Tuner
from ray.tune.registry import register_trainable, validate_trainable

class ClonedClass(type(resto_algo)):  # Inheriting from the type of resto_algo object
    def __new__(cls):
        # Create the instance of the class
        #instance = super(ClonedClass, cls).__new__(cls)
        instance = resto_config.build()  # Initialize the config attribute here

        #checkpoint_path = "/root/ray_results/PPO_2024-10-20_17-36-00/PPO_2_agent_waterworld_c5957_00000_0_2024-10-20_17-36-00/checkpoint_000000"
        #pols = glob(checkpoint_path+"/policies/*")
        #specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}
        #instance.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
        #instance.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())

        return instance

register_trainable("cloned_algo", ClonedClass)
validate_trainable('cloned_algo')

print(f"{get_trainable_cls('PPO')=}")
print(f"{get_trainable_cls('cloned_algo')=}")

get_trainable_cls('PPO')=<class 'ray.rllib.algorithms.ppo.ppo.PPO'>
get_trainable_cls('cloned_algo')=<class '__main__.ClonedClass'>


In [ ]:
a = get_trainable_cls('PPO')
b = get_trainable_cls('PPO')

a is b

#resto_algo = resto_config.build()
#for test_id in range(num_test_agents):
#    train_id = np.random.randint(num_trained_agents)
#    resto_algo.get_policy(f"pursuer_{test_id}").set_weights(specs[f"pursuer_{train_id}"].get_weights())

#resto_algo.get_policy("pursuer_0").set_weights(specs["pursuer_0"].get_weights())
#resto_algo.get_policy("pursuer_1").set_weights(specs["pursuer_1"].get_weights())
#resto_algo.get_policy("pursuer_0").get_weights()

True

In [ ]:
parser = add_rllib_example_script_args(
    default_iters=10,
    default_timesteps=1000000,
    default_reward=300,
)
args = parser.parse_args(args=[])
#args.checkpoint_at_end = True
#args.num_samples = 2
args.num_agents = 2
env = 'waterworld'

register_env(f"{num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=num_agents)))

run_rllib_example_script_experiment(
    #base_config,
    resto_config,
    args=args,
    #stop=stopper,
    trainable="cloned_algo"
    #trainable=resto_algo
)


2024-10-20 21:13:15,041	INFO worker.py:1619 -- Calling ray.init() again after it has already been called.
2024-10-20 21:13:15,060	WARNING algorithm_config.py:4225 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
2024-10-20 21:13:15,061	WARNING algorithm_config.py:4225 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.


2024-10-20 21:13:15,067	WARNING algorithm_config.py:4225 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
2024-10-20 21:13:19,014	WARNING algorithm_config.py:4225 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.
2024-10-20 21:13:19,114	WARNING util.py:61 -- Install gputil for GPU system monitoring.


ValueError: Tracked actor is not managed by this event manager: <TrackedActor 70630161317072633402302813275770675611>

In [5]:
import numpy as np
import sys
from glob import glob
from os import path

import ray
from ray.tune import Trainable, Tuner
from ray.tune.registry import register_trainable, validate_trainable
from ray.rllib.core.rl_module.marl_module import MultiAgentRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.rllib.policy.policy import Policy
from ray.tune.registry import get_trainable_cls, register_env

from pettingzoo.sisl import waterworld_v4




parser = add_rllib_example_script_args(
    default_iters=10,
    default_timesteps=1000000,
    default_reward=300,
)
args = parser.parse_args(args=[])
args.num_env_runners = 10
#env = 'waterworld'
args.num_agents = 2
args.test_agents = 2


checkpoint_path = "/root/ray_results/PPO_2024-10-20_17-36-00/PPO_2_agent_waterworld_c5957_00000_0_2024-10-20_17-36-00/checkpoint_000000"
pols = glob(checkpoint_path+"/policies/*")
specs = {path.basename(p) : Policy.from_checkpoint(p) for p in pols}

register_env(f"{args.num_agents}_agent_env", lambda _: ParallelPettingZooEnv(waterworld_v4.parallel_env(n_pursuers=args.num_agents)))
policies = {f"pursuer_{i}" for i in range(args.num_agents)}


resto_config = (
    get_trainable_cls("PPO")
    .get_default_config()
    .environment(f"{args.num_agents}_agent_env")
    .multi_agent(
        policies=policies,
        policy_mapping_fn=(lambda aid, *args, **kwargs: aid),
    )
    .rl_module(
        rl_module_spec=MultiAgentRLModuleSpec(
            module_specs={p: RLModuleSpec() for p in policies},
        ),
    )
    .evaluation(
        evaluation_interval=1,
    )
)
resto_algo = resto_config.build()
for test_id in range(args.test_agents):
    train_id = np.random.randint(args.num_agents)
    resto_algo.get_policy(f"pursuer_{test_id}").set_weights(specs[f"pursuer_{train_id}"].get_weights())

#resto_algo.evaluate()

#register_trainable("cloned_algo", lambda _: resto_algo)
#run_rllib_example_script_experiment(
#    resto_config,
#    args=args,
#    trainable="cloned_algo"
#)

results = resto_algo.train()

/opt/conda/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/opt/conda/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2024-10-21 00:41:17,212	WARNING algorithm_config.py:4225 -- You have setup a RLModuleSpec (via calling `config.rl_module(...)`), but have not enabled the new API stack. To enable it, call `config.api_stack(enable_rl_module_and_learner=True)`.


/opt/conda/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/conda/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/opt/conda/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppre

In [8]:
results

{'evaluation': {'env_runners': {'episode_reward_max': np.float64(-131.68730216240365),
   'episode_reward_min': np.float64(-266.73560437749757),
   'episode_reward_mean': np.float64(-208.95588420336244),
   'episode_len_mean': np.float64(500.0),
   'episode_media': {},
   'episodes_timesteps_total': 5000,
   'policy_reward_min': {'pursuer_0': np.float64(-90.66629220461637),
    'pursuer_1': np.float64(-203.62428448947136)},
   'policy_reward_max': {'pursuer_0': np.float64(10.859818810065214),
    'pursuer_1': np.float64(-124.92327822959729)},
   'policy_reward_mean': {'pursuer_0': np.float64(-47.40599192993528),
    'pursuer_1': np.float64(-161.54989227342708)},
   'custom_metrics': {},
   'hist_stats': {'episode_reward': [np.float64(-256.2627302966058),
     np.float64(-248.82995988943728),
     np.float64(-177.90856596823969),
     np.float64(-266.73560437749757),
     np.float64(-131.68730216240365),
     np.float64(-224.42221752494717),
     np.float64(-136.19221133120277),
     np

In [19]:
results = resto_algo.train()
print(f"{results['training_iteration']=}")
print(f"{results['env_runners']['num_episodes']=}")
print(f"{results['env_runners']['episode_return_mean']=}")
# 12000

results['training_iteration']=3
results['env_runners']['num_episodes']=8
results['env_runners']['episode_return_mean']=np.float64(-190.84602312225365)


In [21]:
results = resto_algo.train()
print(f"{results['training_iteration']=}")
print(f"{results['env_runners']['num_episodes']=}")
print(f"{results['env_runners']['episode_return_mean']=}")
print(f"{results['env_runners']['episodes_timesteps_total']=}")

results['training_iteration']=4
results['env_runners']['num_episodes']=8
results['env_runners']['episode_return_mean']=np.float64(-191.70060061384294)
results['env_runners']['episodes_timesteps_total']=16000


In [20]:

# timesteps
results['env_runners']['episodes_timesteps_total']


#reward_history.append(results['episode_reward_mean'])
#moving_average_reward = np.mean(reward_history)


12000